## Dependencies

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from tensorflow.keras import Model, optimizers
from tensorflow.keras.layers import Lambda, Input, Dense, Dropout, Concatenate, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback

def seed_everything(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
hold_out = pd.read_csv('/kaggle/input/googleqa-dataset/hold-out.csv')
train = hold_out[hold_out['set'] == 'train']
validation = hold_out[hold_out['set'] == 'validation']
test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Train samples: %s' % len(train))
print('Validation samples: %s' % len(validation))
print('Test samples: %s' % len(test))
display(train.head())

Train samples: 4863
Validation samples: 1216
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written,set
0,6148,Important non-technical course for programmers?,What kind of non-technical training course do ...,Louis Rhys,https://programmers.stackexchange.com/users/8486,Business\n\nThe biggest problem I've seen with...,Ryan Hayes,https://programmers.stackexchange.com/users/1521,http://programmers.stackexchange.com/questions...,TECHNOLOGY,...,1.000000,0.666667,1.0,1.0,0.933333,0.000000,0.000000,1.0,0.888889,train
1,3971,Water : Aquatic :: Sand : xxx?,Just as aquatic is to water and aerial is to a...,coleopterist,https://english.stackexchange.com/users/23608,"Well, fancy words for ‘sandy’ are arenarious a...",tchrist,https://english.stackexchange.com/users/2085,http://english.stackexchange.com/questions/938...,CULTURE,...,1.000000,1.000000,1.0,1.0,1.000000,0.000000,0.000000,1.0,0.888889,train
2,4367,"Learning the musical concepts in the book ""Göd...","I am reading the book ""Gödel, Escher, Bach"", i...",Otavio Macedo,https://music.stackexchange.com/users/12666,I agree with luser droog's answer.\n\nMy under...,luser droog,https://music.stackexchange.com/users/1344,http://music.stackexchange.com/questions/22216...,LIFE_ARTS,...,1.000000,0.666667,1.0,1.0,0.900000,0.333333,0.333333,1.0,0.777778,train
3,6605,HowTo: Add Class to Sidebar Widget List-Items,The newest version of Bootstrap (v3.0) adds a ...,sleeper,https://wordpress.stackexchange.com/users/12116,Update: I fixed the debug warning and everythi...,sleeper,https://wordpress.stackexchange.com/users/12116,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,...,0.777778,0.555556,1.0,1.0,0.733333,0.500000,0.000000,0.5,0.888889,train
4,6522,MVC4 jQuery UI does not work,I cannot get jQuery UI working in my ASP.NET M...,Petr,https://stackoverflow.com/users/958557,"In the layout.cshtml view, move \n\n@Scripts.R...",Yuri Morales,https://stackoverflow.com/users/1459163,http://stackoverflow.com/questions/13112519/mv...,STACKOVERFLOW,...,1.000000,0.666667,1.0,1.0,1.000000,0.000000,1.000000,0.0,1.000000,train


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

In [4]:
class SpearmanRhoCallback(Callback):
    def __init__(self, training_data, validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_train = self.model.predict(self.x)
        rho_train = np.mean([spearmanr(self.y[:, ind], y_pred_train[:, ind] + np.random.normal(0, 1e-7, y_pred_train.shape[0])).correlation for ind in range(y_pred_train.shape[1])])
        
        y_pred_val = self.model.predict(self.x_val)
        rho_val = np.mean([spearmanr(self.y_val[:, ind], y_pred_val[:, ind] + np.random.normal(0, 1e-7, y_pred_val.shape[0])).correlation for ind in range(y_pred_val.shape[1])])
        print('\rTrain spearman-rho: %.4f Validation spearman-rho: %.4f' % (rho_train, rho_val))
        
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

## Train/validation sets

In [5]:
# Train features
X_train_title = train['question_title']
X_train_body = train['question_body']
X_train_answer = train['answer']

X_train = [X_train_title, X_train_body, X_train_answer]
Y_train = train[target_cols].values

# Validation features
X_valid_title = validation['question_title']
X_valid_body = validation['question_body']
X_valid_answer = validation['answer']

X_valid = [X_valid_title, X_valid_body, X_valid_answer]
Y_valid = validation[target_cols].values

print('Train samples: %d' % len(Y_train))
print('Validation samples: %d' % len(Y_valid))

Train samples: 4863
Validation samples: 1216


# Model parameters

In [6]:
EPOCHS = 30
BATCH_SIZE = 64
LEARNING_RATE = 3e-4
EMBEDDDING_SIZE = 512
N_CLASS = len(target_cols)
ES_PATIENCE = 3
RLROP_PATIENCE = 2
DECAY_DROP = 0.5
module_url = '../input/universalsentenceencodermodels/universal-sentence-encoder-models/use-qa'
model_path = '../working/use_baseline.h5'

# Model

In [7]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)
use_embed = hub.load(module_url)

def USEEmbedding(x):
    return use_embed(tf.squeeze(tf.cast(x, tf.string)))

In [8]:
input_title = Input(shape=(1,), dtype=tf.string, name='input_title')
embedding_title = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_title)

input_body = Input(shape=(1,), dtype=tf.string, name='input_body')
embedding_body = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_body)

input_answer = Input(shape=(1,), dtype=tf.string, name='input_answer')
embedding_answer = Lambda(USEEmbedding, output_shape=(EMBEDDDING_SIZE,))(input_answer)

x = Concatenate()([embedding_title, embedding_body, embedding_answer])
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(N_CLASS, activation='sigmoid', name='output')(x)
model = Model(inputs=[input_title, input_body, input_answer], outputs=[output])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_title (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_body (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_answer (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 512)          0           input_title[0][0]                
______________________________________________________________________________________________

# Train model

In [9]:
optimizer = optimizers.Adam(LEARNING_RATE)
callback_list = [es, rlrop, SpearmanRhoCallback(training_data=(X_train, Y_train), validation_data=(X_valid, Y_valid))]
model.compile(optimizer=optimizer, loss='binary_crossentropy')

history = model.fit(X_train, Y_train, 
                    validation_data=(X_valid, Y_valid), 
                    callbacks=callback_list, 
                    epochs=EPOCHS, 
                    verbose=2).history

Train on 4863 samples, validate on 1216 samples
Epoch 1/30
Train spearman-rho: 0.2319 Validation spearman-rho: 0.2228
4863/4863 - 248s - loss: 0.4948 - val_loss: 0.4091
Epoch 2/30
Train spearman-rho: 0.3135 Validation spearman-rho: 0.2988
4863/4863 - 125s - loss: 0.4059 - val_loss: 0.3920
Epoch 3/30
Train spearman-rho: 0.3537 Validation spearman-rho: 0.3330
4863/4863 - 125s - loss: 0.3942 - val_loss: 0.3843
Epoch 4/30
Train spearman-rho: 0.3759 Validation spearman-rho: 0.3488
4863/4863 - 124s - loss: 0.3879 - val_loss: 0.3799
Epoch 5/30
Train spearman-rho: 0.3912 Validation spearman-rho: 0.3586
4863/4863 - 124s - loss: 0.3837 - val_loss: 0.3771
Epoch 6/30
Train spearman-rho: 0.4033 Validation spearman-rho: 0.3652
4863/4863 - 124s - loss: 0.3802 - val_loss: 0.3752
Epoch 7/30
Train spearman-rho: 0.4135 Validation spearman-rho: 0.3706
4863/4863 - 124s - loss: 0.3783 - val_loss: 0.3739
Epoch 8/30
Train spearman-rho: 0.4214 Validation spearman-rho: 0.3745
4863/4863 - 124s - loss: 0.3757 - v

#### Save model

In [10]:
model.save_weights(model_path)

# Evaluation

In [11]:
preds_train = model.predict(X_train)
preds_val = model.predict(X_valid)

rho_train = np.mean([spearmanr(Y_train[:, ind], preds_train[:, ind] + np.random.normal(0, 1e-7, preds_train.shape[0])).correlation for ind in range(preds_train.shape[1])])
rho_val = np.mean([spearmanr(Y_valid[:, ind], preds_val[:, ind] + np.random.normal(0, 1e-7, preds_val.shape[0])).correlation for ind in range(preds_val.shape[1])])

print('Train spearman-rho: %.3f' % rho_train)
print('Validation spearman-rho: %.3f' % rho_val)

Train spearman-rho: 0.491
Validation spearman-rho: 0.391


# Make predictions on test

In [12]:
# Test features
X_test_title = test['question_title']
X_test_body = test['question_body']
X_test_answer = test['answer']

X_test = [X_test_title, X_test_body, X_test_answer]
Y_test = model.predict(X_test)

In [13]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.908022,0.694427,0.322958,0.712988,0.573246,0.607208,0.611984,0.576663,0.242682,...,0.892192,0.866505,0.587616,0.931210,0.940966,0.793737,0.050508,0.076169,0.759590,0.903942
1,46,0.873666,0.575491,0.001503,0.633873,0.840183,0.916526,0.575993,0.488513,0.124211,...,0.745943,0.938297,0.624243,0.962416,0.972376,0.854162,0.950555,0.137652,0.100155,0.887197
2,70,0.898049,0.658574,0.013796,0.729516,0.898561,0.936532,0.601119,0.484168,0.384925,...,0.845650,0.908437,0.610735,0.955214,0.953397,0.818698,0.124401,0.126407,0.806248,0.889717
3,132,0.841756,0.442662,0.020872,0.692267,0.810821,0.825974,0.563518,0.473811,0.399421,...,0.713143,0.930135,0.689350,0.950585,0.971980,0.876865,0.722110,0.148133,0.668399,0.867175
4,200,0.943154,0.537223,0.009712,0.833631,0.844439,0.945931,0.620578,0.577057,0.260023,...,0.787583,0.949776,0.657096,0.979343,0.978914,0.865545,0.490860,0.194425,0.515519,0.917280
